In [1]:
import configparser
config = configparser.ConfigParser()
config.read('config.ini')
Username = config['Credentials']['Username']
PrivateKey = config['Credentials']['PrivateKey']

**Getting total number of DefaultCollateral contracts**

In [3]:
from web3 import Web3
import json

infura_url = "https://holesky.infura.io/v3/a77f7fa2b1684b61b8de38395308ae51"
web3 = Web3(Web3.HTTPProvider(infura_url))

# Check if connected to the network
if not web3.is_connected():
    raise Exception("Failed to connect to Ethereum network")

# factory_address = "0x1BC8FCFbE6Aa17e4A7610F51B888f34583D202Ec"
factory_address = "0x7224eeF9f38E9240beA197970367E0A8CBDFDD8B"

# Load ABIs
with open("DefaultCollateralABI.json", "r") as f:
    collateral_abi = json.load(f)

with open("DefaultCollateralFactoryHolesky.json", "r") as f:
    factory_abi = json.load(f)

# Initialize factory contract
factory_contract = web3.eth.contract(address=factory_address, abi=factory_abi)

def index_collateral_contracts():
    # Get the total number of DefaultCollateral contracts
    total_entities = factory_contract.functions.totalEntities().call()
    print(total_entities)
    if total_entities == 0:
        print("No DefaultCollateral contracts found.")
        return []

    collateral_addresses = [factory_contract.functions.entity(i).call() for i in range(total_entities)]
    
    print(f"Found {len(collateral_addresses)} DefaultCollateral contracts")

    collateral_info = []
    
    for address in collateral_addresses:
        print(f"Contract address -  {address}")
           
    return collateral_info

collateral_contracts = index_collateral_contracts()


43
Found 43 DefaultCollateral contracts
Contract address -  0x23E98253F372Ee29910e22986fe75Bb287b011fC
Contract address -  0x4d955DDB44514bdC9CEDD966109D246FD44f6e12
Contract address -  0x3d9c7A2c18f46f170EC63aa53bf80fe9F7b67608
Contract address -  0x69Ce5e58B3c19Fe94c6d2D106AD95337730b12ff
Contract address -  0x25De43b94b1DF44aC237Cb9B05868755238F2858
Contract address -  0xA9AF5Fa1a55C29a7f75f9d446F421C1F18A86407
Contract address -  0x18AAB2F2dd4732042c22569620fcfa87E4cC7FB3
Contract address -  0x32F12fAc562a6674D34012e3840CE0511444dee1
Contract address -  0x8aE47d8848B88E667F2AD3b0DcAa966182Ad19d6
Contract address -  0xD682cf1dcfb3B0221Cbe4B708A0d2Ef70Fa74e74
Contract address -  0xC9DE51D8E7553fdbb5f2821B7e3304B99413d6ad
Contract address -  0x70350d8e80c29a8a779630194b5246A73944bc5B
Contract address -  0xdEfb41DD3EBEfbA6d463b13a8447DF20D0051E37
Contract address -  0x6429e49c46a2f4A1F7C81719f91A0fd5E0377cA1
Contract address -  0x49502187FcD748818a9d06F29765d0C105FACC33
Contract addres

**Implementing basic withdraw and deposit function**

In [4]:
from web3 import Web3
import json
import time
from requests.exceptions import HTTPError

infura_url = "https://holesky.infura.io/v3/a77f7fa2b1684b61b8de38395308ae51"
web3 = Web3(Web3.HTTPProvider(infura_url))

if not web3.is_connected():
    raise Exception("Failed to connect to Ethereum network")
else:
    print("Connected to network")

#wstETH address
collateral_address = "0xC329400492c6ff2438472D4651Ad17389fCb843a"  
with open("DefaultCollateralABI.json", "r") as f:
    collateral_abi = json.load(f)

collateral_contract = web3.eth.contract(address=web3.to_checksum_address(collateral_address), abi=collateral_abi)

# Acc details 
account_address = Username
private_key = PrivateKey 


def get_increased_gas_price(multiplier=1.2, retries=3, delay=2):
    for attempt in range(retries):
        try:
            gas_price = web3.eth.gas_price
            return int(gas_price * multiplier)
        except HTTPError as e:
            if e.response.status_code == 429:
                print("Rate limit reached. Retrying in {} seconds...".format(delay))
                time.sleep(delay)
                delay *= 2  # Exponential backoff
            else:
                raise
    raise Exception("Exceeded maximum retries for getting gas price")


def deposit_collateral(recipient, amount):
   
    txn = collateral_contract.functions.deposit(recipient, amount).build_transaction({
        'from': account_address,
        'nonce': web3.eth.get_transaction_count(account_address),
        'gas': 200000, 
        'gasPrice': web3.eth.gas_price
    })
    signed_txn = web3.eth.account.sign_transaction(txn, private_key=private_key)
    
    tx_hash = web3.eth.send_raw_transaction(signed_txn.raw_transaction)
    print(f"Deposit transaction sent, tx hash: {tx_hash.hex()}")
    
    receipt = web3.eth.wait_for_transaction_receipt(tx_hash)
    print("Deposit transaction receipt:", receipt)
    
def withdraw_collateral(recipient, amount):
    """
    Build, sign, and send a transaction to withdraw a given amount.
    The withdraw function burns the collateral tokens from your account and transfers the underlying asset.
    """
    nonce = web3.eth.get_transaction_count(account_address, "pending")
    gas_price = web3.eth.gas_price
    increased_gas_price = get_increased_gas_price()
    txn = collateral_contract.functions.withdraw(recipient, amount).build_transaction({
        'from': account_address,
        'nonce': nonce,
        'gas': 200000,  
        'gasPrice': increased_gas_price
    })
    
    signed_txn = web3.eth.account.sign_transaction(txn, private_key=private_key)
    tx_hash = web3.eth.send_raw_transaction(signed_txn.raw_transaction)
    print(f"Withdraw transaction sent, tx hash: {tx_hash.hex()}")
    receipt = web3.eth.wait_for_transaction_receipt(tx_hash)
    print("Withdraw transaction receipt:", receipt)

if __name__ == "__main__":
    deposit_amount = 1000  
    recipient_address = account_address  
    deposit_collateral(recipient_address, deposit_amount)
    
    withdraw_amount = 500
    withdraw_collateral(recipient_address, withdraw_amount)


Connected to network
Deposit transaction sent, tx hash: 6a89e398abe2fd25d93a4b154a0cb4bb56feb07a3061d099afa74479a4cc7c99
Deposit transaction receipt: AttributeDict({'blockHash': HexBytes('0x3cb11ca2f2e6ffde70dbbab43e5b9089c5f3398a88bee554f3308538b3108fcf'), 'blockNumber': 3384441, 'contractAddress': None, 'cumulativeGasUsed': 1391128, 'effectiveGasPrice': 1000015, 'from': '0xa674541764CC1DAc9C10A17EDc9CC2921816140b', 'gasUsed': 21584, 'logs': [], 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'), 'status'

**Implementing withdraw and deposit function with basic conversion logic**

In [ ]:
from web3 import Web3
import json
import time

infura_url = "https://holesky.infura.io/v3/a77f7fa2b1684b61b8de38395308ae51"
web3 = Web3(Web3.HTTPProvider(infura_url))

if not web3.is_connected():
    raise Exception("Failed to connect to the Ethereum network")
else:
    print("Connected to network")

collateral_address = "0xC329400492c6ff2438472D4651Ad17389fCb843a" 

# Load the DefaultCollateral ABI
with open("DefaultCollateralABI.json", "r") as f:
    collateral_abi = json.load(f)

collateral_contract = web3.eth.contract(address=web3.to_checksum_address(collateral_address), abi=collateral_abi)

# Acc details 
account_address = Username
private_key = PrivateKey 


# assuming a simple 1:1 conversion (I deposit 1 token I get 1 wsteth token).
deposit_conversion_rate = 1.0
withdraw_conversion_rate = 1.0

def calculate_deposit_output(input_amount, conversion_rate=1.0):
    return input_amount * conversion_rate

def calculate_withdraw_output(input_amount, conversion_rate=1.0):
    return input_amount * conversion_rate

def deposit_collateral(recipient, input_amount, conversion_rate=1.0):
    expected_output = calculate_deposit_output(input_amount, conversion_rate)
    print(f"Expected deposit output : {expected_output}")

    # Build the transaction
    nonce = web3.eth.get_transaction_count(account_address, "pending")
    txn = collateral_contract.functions.deposit(recipient, input_amount).build_transaction({
        'from': account_address,
        'nonce': nonce,
        'gas': 300000,  
        'gasPrice': web3.eth.gas_price
    })
    
    signed_txn = web3.eth.account.sign_transaction(txn, private_key=private_key)
    tx_hash = web3.eth.send_raw_transaction(signed_txn.raw_transaction)
    print(f"Deposit transaction sent, tx hash: {tx_hash.hex()}")
    
    receipt = web3.eth.wait_for_transaction_receipt(tx_hash)
    print("Deposit transaction receipt:", receipt)
    return receipt

def withdraw_collateral(recipient, input_amount, conversion_rate=1.0):

    expected_output = calculate_withdraw_output(input_amount, conversion_rate)
    print(f"Expected withdraw output : {expected_output}")

    nonce = web3.eth.get_transaction_count(account_address, "pending")
    txn = collateral_contract.functions.withdraw(recipient, input_amount).build_transaction({
        'from': account_address,
        'nonce': nonce,
        'gas': 300000,  
        'gasPrice': web3.eth.gas_price
    })
    
    signed_txn = web3.eth.account.sign_transaction(txn, private_key=private_key)
    
    tx_hash = web3.eth.send_raw_transaction(signed_txn.raw_transaction)
    print(f"Withdraw transaction sent, tx hash: {tx_hash.hex()}")
    
    receipt = web3.eth.wait_for_transaction_receipt(tx_hash)
    print("Withdraw transaction receipt:", receipt)
    return receipt

if __name__ == "__main__":
    recipient = account_address  
    deposit_amount = 1000         
    withdraw_amount = 500         

    print("=== Deposit Operation ===")
    deposit_collateral(recipient, deposit_amount, deposit_conversion_rate)
    
    # delay to avoid nonce conflicts
    time.sleep(10)
    
    print("=== Withdraw Operation ===")
    withdraw_collateral(recipient, withdraw_amount, withdraw_conversion_rate)


Connected to network
=== Deposit Operation ===
Expected deposit output : 1000.0
Deposit transaction sent, tx hash: b9b854871b2139cd14ba23542161d6387c772fd58d92809e13873299973cbba7
Deposit transaction receipt: AttributeDict({'blockHash': HexBytes('0x7dae4be4d0dc90eaab64faa098d9c42be1c5eacd72dc7506f7352f15a2618270'), 'blockNumber': 3384477, 'contractAddress': None, 'cumulativeGasUsed': 1426884, 'effectiveGasPrice': 1050012, 'from': '0xa674541764CC1DAc9C10A17EDc9CC2921816140b', 'gasUsed': 21584, 'logs': [], 'logsBloom': HexBytes('0x000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000